In [1]:
import os
import json

SOURCE_DIR = os.path.join('data', 'temp')
os.listdir(SOURCE_DIR)

['gemma_evaluation_data.json']

In [2]:
def json_load(file_path):
    with open(file_path, 'r') as f:
        data_dict = json.load(f)
    return data_dict

def json_dump(file_path, data):
    with open(file_path, 'w') as f:
        print("write result to: " + file_path)
        json.dump(data, f, indent=1, ensure_ascii=False)

In [3]:
gemma_eval_file_path = os.path.join(SOURCE_DIR, 'gemma_evaluation_data.json')
data = json_load(gemma_eval_file_path)
data[0]

{'reference_answer': {'qid': '1',
  'stem': '常見針灸配穴法中,所指的「四關穴」,為下列何穴位之組合?',
  'A': '上星、日月',
  'B': '合谷、太衝',
  'C': '內關、外關',
  'D': '上關、下關'},
 'reference_context': ['1.常見針灸配穴法中，所指的「四關穴」，為下列何穴位之組合？\n\xa0\nA.上星、日月\nB.合谷、太衝\nC.內關、外關\nD.上關、下關'],
 'response': {'qid': 1,
  'stem': '常見針灸配穴法中，所指的「四關穴」，為下列何穴位之組合？',
  'A': '上星、日月',
  'B': '合谷、太衝',
  'C': '內關、外關',
  'D': '上關、下關'}}

In [4]:
num_succ = 0
num_fail = 0
total = len(data)
fail_case = []
for example in data:
    label_stem = example['reference_answer']['stem']
    pred_stem = example['response']['stem']
    if label_stem == pred_stem:
        num_succ+=1
    else:
        num_fail+=1
        fail_case.append(
            {
                'label': label_stem,
                'pred': pred_stem,
            }
        )

print(f"succ_rate: {num_succ/total:.2f}")


succ_rate: 0.00


In [5]:
fail_case[0]

{'label': '常見針灸配穴法中,所指的「四關穴」,為下列何穴位之組合?',
 'pred': '常見針灸配穴法中，所指的「四關穴」，為下列何穴位之組合？'}

# normalize exact match

In [16]:
import re

def normalize_text(s: str) -> str:
    """簡單的 normalization: 小寫化、去標點、去多餘空白"""
    # 全部轉小寫
    s = s.lower()
    # 移除所有標點符號（只保留中英文、數字、空白）
    s = re.sub(r"[^0-9a-z\u4e00-\u9fff\s]", "", s)
    # 去掉多餘空白
    s = " ".join(s.split())
    return s

In [20]:
num_succ = 0
num_fail = 0
total = len(data)
fail_case = []
for example in data:
    qid = example['reference_answer']['qid']
    label_stem = normalize_text(example['reference_answer']['stem'])
    pred_stem = normalize_text(example['response']['stem'])
    if label_stem == pred_stem:
        num_succ+=1
    else:
        num_fail+=1
        fail_case.append(
            {
                'qid': qid,
                'label': label_stem,
                'pred': pred_stem,
            }
        )

print(f"succ_rate: {num_succ/total:.2f}")


succ_rate: 0.75


In [21]:
fail_case[:5]

[{'qid': '2',
  'label': '依靈樞經脈記載其直者從巔入絡腦還出別下項循肩膊內挾脊抵腰中指下列 何經的循行內容',
  'pred': '依靈樞經脈記載其直者從巔入絡腦還出別下項循肩膊內挾脊抵腰中指下列何經的循行內容'},
 {'qid': '9',
  'label': '下列穴位屬三焦經與膽經的交會穴共有幾個1臑會 2顴髎 3秉風 4聽宮 5耳門',
  'pred': '下列穴位屬三焦經與膽經的交會穴共有幾個臑會 顴髎 秉風 聽宮 耳門'},
 {'qid': '10',
  'label': '有關手少陰心經的通里穴下列敘述共幾項正確1手少陰心經的郄穴 2馬丹陽天星十二穴 之一 3距離靈道穴五分 4配內關治心絞痛',
  'pred': '有關手少陰心經的通里穴下列敘述共幾項正確手少陰心經的郄穴 馬丹陽天星十二穴之一 距離靈道穴五分 配內關治心絞痛'},
 {'qid': '16',
  'label': '有關本神穴的敘述下列選項何者正確1上入髮際一寸從神庭外開二寸處 2曲差旁一寸五分 3可治頭痛目眩等 4配百會人中十宣治中風不省人事',
  'pred': '有關本神穴的敘述下列選項何者正確上入髮際一寸從神庭外開二寸處 曲差旁一寸五分 可治頭痛目眩等 配百會人中十宣治中風不省人事'},
 {'qid': '28',
  'label': '48歲女性患者診斷為類風濕性關節炎已2年主訴四肢肩膀至手指遊走性疼痛夜間及氣候變化 疼痛加重下列敘述何者錯誤',
  'pred': '2848歲女性患者診斷為類風濕性關節炎已2年主訴四肢肩膀至手指遊走性疼痛夜間及氣候變化疼痛加重下列敘述何者錯誤'}]

In [22]:
import unicodedata
import re
from typing import List, Tuple

def _remove_punctuation(text: str) -> str:
    # 移除所有 Unicode 標點 (類別以 'P' 開頭)
    return "".join(ch for ch in text if not unicodedata.category(ch).startswith('P'))

def _remove_all_whitespace(text: str) -> str:
    # \s 包含空格、tab、換行、全形空白等各種空白
    return re.sub(r"\s+", "", text)

def normalize_text(
    s: str,
    *,
    use_nfkc: bool = True,
    lowercase: bool = True,
    remove_space: bool = True,
    remove_punct: bool = True
) -> str:
    """
    常見 normalized exact match 的正規化步驟：
    - NFKC：全形→半形、相容字元折疊（建議開）
    - lowercase：小寫化（對中文無影響，但混合文本時建議開）
    - remove_space：移除所有空白（做比對時建議開）
    - remove_punct：移除所有標點（建議開，以避免中英標點差異）
    """
    if s is None:
        return ""
    if use_nfkc:
        s = unicodedata.normalize("NFKC", s)
    if lowercase:
        s = s.lower()
    if remove_punct:
        s = _remove_punctuation(s)
    if remove_space:
        s = _remove_all_whitespace(s)
    return s

In [1]:
import unicodedata
def unicode_normalize(txt):
    s = unicodedata.normalize("NFKC", txt)
    return s

In [2]:
example_text = '①臑會　②顴髎　③秉風　④聽宮　⑤耳門'
print(unicode_normalize(example_text))

1臑會 2顴髎 3秉風 4聽宮 5耳門


In [27]:
all_case = []
num_succ = 0
num_fail = 0
total = len(data)
fail_case = []
for example in data:
    qid = example['reference_answer']['qid']
    label_stem = example['reference_answer']['stem']
    pred_stem = example['response']['stem']
    nlabel_stem = normalize_text(label_stem)
    npred_stem = normalize_text(pred_stem)
    
    if nlabel_stem == npred_stem:
        num_succ+=1
    else:
        num_fail+=1
        fail_case.append(
            {
                'qid': qid,
                'label': label_stem,
                'pred': pred_stem,
                'nlabel': nlabel_stem,
                'npred': npred_stem,
            }
        )
    all_case.append(
            {
                'qid': qid,
                'label': label_stem,
                'pred': pred_stem,
                'nlabel': nlabel_stem,
                'npred': npred_stem,
            }
    )

print(f"succ_rate: {num_succ/total:.2f}")


succ_rate: 0.99


In [24]:
fail_case

[{'qid': '28',
  'label': '48歲女性患者診斷為類風濕性關節炎已2年主訴四肢肩膀至手指遊走性疼痛夜間及氣候變化疼痛加重下列敘述何者錯誤',
  'pred': '2848歲女性患者診斷為類風濕性關節炎已2年主訴四肢肩膀至手指遊走性疼痛夜間及氣候變化疼痛加重下列敘述何者錯誤'}]

In [28]:
all_case[0]

{'qid': '1',
 'label': '常見針灸配穴法中,所指的「四關穴」,為下列何穴位之組合?',
 'pred': '常見針灸配穴法中，所指的「四關穴」，為下列何穴位之組合？',
 'nlabel': '常見針灸配穴法中所指的四關穴為下列何穴位之組合',
 'npred': '常見針灸配穴法中所指的四關穴為下列何穴位之組合'}

In [29]:
all_case[1]

{'qid': '2',
 'label': '依《靈樞.經脈》記載,「其直者,從巔入絡腦,還出別下項,循肩膊內,挾脊抵腰中」,指下列 何經的循行內容?',
 'pred': '依《靈樞．經脈》記載，「其直者，從巔入絡腦，還出別下項，循肩膊內，挾脊抵腰中」，指下列何經的循行內容？',
 'nlabel': '依靈樞經脈記載其直者從巔入絡腦還出別下項循肩膊內挾脊抵腰中指下列何經的循行內容',
 'npred': '依靈樞經脈記載其直者從巔入絡腦還出別下項循肩膊內挾脊抵腰中指下列何經的循行內容'}

In [30]:
all_case[2]

{'qid': '3',
 'label': '依《靈樞.經脈》所記載:「是主筋所生病者,痔、瘧、狂、癲疾、頭顖、項痛,目黃、淚出,鼽 衄,項、背、腰、尻、膕、腨、腳皆痛,小趾不用」,指何經的病證內容?',
 'pred': '依《靈樞．經脈》所記載：「是主筋所生病者，痔、瘧、狂、癲疾、頭顖、項痛，目黃、淚出，鼽\n衄，項、背、腰、尻、膕、腨、腳皆痛，小趾不用」，指何經的病證內容？',
 'nlabel': '依靈樞經脈所記載是主筋所生病者痔瘧狂癲疾頭顖項痛目黃淚出鼽衄項背腰尻膕腨腳皆痛小趾不用指何經的病證內容',
 'npred': '依靈樞經脈所記載是主筋所生病者痔瘧狂癲疾頭顖項痛目黃淚出鼽衄項背腰尻膕腨腳皆痛小趾不用指何經的病證內容'}

In [33]:
all_case[8]

{'qid': '9',
 'label': '下列穴位屬三焦經與膽經的交會穴共有幾個?1臑會 2顴髎 3秉風 4聽宮 5耳門',
 'pred': '下列穴位屬三焦經與膽經的交會穴共有幾個？①臑會\u3000②顴髎\u3000③秉風\u3000④聽宮\u3000⑤耳門',
 'nlabel': '下列穴位屬三焦經與膽經的交會穴共有幾個1臑會2顴髎3秉風4聽宮5耳門',
 'npred': '下列穴位屬三焦經與膽經的交會穴共有幾個1臑會2顴髎3秉風4聽宮5耳門'}

In [32]:
all_case[9]

{'qid': '10',
 'label': '有關手少陰心經的「通里穴」,下列敘述共幾項正確?1手少陰心經的郄穴 2馬丹陽天星十二穴 之一 3距離靈道穴五分 4配內關,治心絞痛',
 'pred': '有關手少陰心經的「通里穴」，下列敘述共幾項正確？①手少陰心經的郄穴\u3000②馬丹陽天星十二穴之一\u3000③距離靈道穴五分\u3000④配內關，治心絞痛',
 'nlabel': '有關手少陰心經的通里穴下列敘述共幾項正確1手少陰心經的郄穴2馬丹陽天星十二穴之一3距離靈道穴五分4配內關治心絞痛',
 'npred': '有關手少陰心經的通里穴下列敘述共幾項正確1手少陰心經的郄穴2馬丹陽天星十二穴之一3距離靈道穴五分4配內關治心絞痛'}

In [34]:
fail_case

[{'qid': '28',
  'label': '48歲女性患者,診斷為類風濕性關節炎已2年,主訴四肢肩膀至手指遊走性疼痛,夜間及氣候變化 疼痛加重。下列敘述何者錯誤?',
  'pred': '28.48歲女性患者，診斷為類風濕性關節炎已2年，主訴四肢肩膀至手指遊走性疼痛，夜間及氣候變化疼痛加重。下列敘述何者錯誤？',
  'nlabel': '48歲女性患者診斷為類風濕性關節炎已2年主訴四肢肩膀至手指遊走性疼痛夜間及氣候變化疼痛加重下列敘述何者錯誤',
  'npred': '2848歲女性患者診斷為類風濕性關節炎已2年主訴四肢肩膀至手指遊走性疼痛夜間及氣候變化疼痛加重下列敘述何者錯誤'}]